<a href="https://colab.research.google.com/github/DavidSchneider47/National-TOD-Atlas/blob/main/Station_Area_GeoJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import some libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

### **Createa a GeoJSON File with a 1/4 Mile Buffer Around Station Lon/Lat Coordinates**

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

# Reload the dataset due to code execution state reset
file_path = 'stations_clean_final3.csv'
data = pd.read_csv(file_path)

# Ensure column names are not more than 10 characters long
data.columns = [col[:10] for col in data.columns]

# Extracting longitude and latitude from the 'coordinates' column
# Splitting the 'coordinates' column into two new columns 'Lat' and 'Lon'
data[['Lat', 'Lon']] = data['coordinate'].str.extract(r'\(([^,]+),\s?([^)]+)\)')

# Convert Lat and Lon to numeric, handling errors for non-numeric values
data['Lat'] = pd.to_numeric(data['Lat'], errors='coerce')
data['Lon'] = pd.to_numeric(data['Lon'], errors='coerce')

# Dropping rows with missing or invalid coordinates
geo_df = data.dropna(subset=['Lat', 'Lon'])

# Create a GeoDataFrame from the data
geometry = [Point(xy) for xy in zip(geo_df.Lon, geo_df.Lat)]
geo_df = gpd.GeoDataFrame(geo_df, geometry=geometry)

# Define the CRS (coordinate reference system)
geo_df.crs = CRS("EPSG:4326")  # This is the CRS for latitude/longitude
geo_df = geo_df.to_crs("EPSG:3857")  # Converting to a CRS that uses meters as units (Web Mercator)

# Create a buffer of 1/4 mile around each point
buffer = geo_df.buffer(402.336)  # 1/4 mile is about 402.336 meters

# Replace 'geometry' with buffer
geo_df.set_geometry(buffer, inplace=True)

# Optionally, convert back to latitude/longitude for compatibility with other tools
geo_df = geo_df.to_crs("EPSG:4326")

# Save the result to a new shapefile or GeoJSON
geo_df.to_file('buffered_stations.shp')


### **Export the File for Inspection and Review some Information**

In [ ]:
from google.colab import files
files.download('buffered_stations.shp')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Reading the buffered stations shapefile to view the column headers and first five rows

# Path to the shapefile
shapefile_path = 'buffered_stations.shp'

# Reading the shapefile using GeoPandas
buffered_stations = gpd.read_file(shapefile_path)

# Displaying the column headers and the first five rows
buffered_stations.head()


,NTD ID,Agency Nam,Reporter T,Reporting,Primary Mo,Facility I,Facility T,Facility N,Street Add,City,...,full_addre,lon_digits,lat_digits,coordinate,County,Category,Definition,Lat,Lon,geometry
0,1,King County Department of Metro Transit,Full Reporter,Urban,Streetcar Rail,1849,Simple At-Grade Platform Station,FHSC: S JACKSON ST BETWEEN 1ST AVE S AND OCCID...,S Jackson and Occidental Ave S,Seattle,...,"S Jackson and Occidental Ave S, Seattle, WA ,9...",4,5,"(47.5878163, -122.333248)",King County,6,Modern Streetcar,47.587816,-122.333248,"POLYGON ((-122.32963 47.58782, -122.32965 47.5..."
1,1,King County Department of Metro Transit,Full Reporter,Urban,Streetcar Rail,1850,Simple At-Grade Platform Station,FHSC: S JACKSON ST BETWEEN 5TH AVE S AND 6TH A...,S Jackson and 6th Ave S,Seattle,...,"S Jackson and 6th Ave S, Seattle, WA ,98104.0",4,4,"(47.5725498, -122.3259876)",King County,6,Modern Streetcar,47.572550,-122.325988,"POLYGON ((-122.32237 47.57255, -122.32239 47.5..."
2,1,King County Department of Metro Transit,Full Reporter,Urban,Streetcar Rail,1851,Simple At-Grade Platform Station,FHSC: S JACKSON ST BETWEEN 7TH AVE S AND 8TH A...,S Jackson and 8th Ave S,Seattle,...,"S Jackson and 8th Ave S, Seattle, WA ,98104.0",4,4,"(47.6061389, -122.3328481)",King County,6,Modern Streetcar,47.606139,-122.332848,"POLYGON ((-122.32923 47.60614, -122.32925 47.6..."
3,1,King County Department of Metro Transit,Full Reporter,Urban,Streetcar Rail,1852,Simple At-Grade Platform Station,FHSC: S JACKSON ST BETWEEN 12TH AVE S AND 14TH...,S Jackson and 14th Ave S,Seattle,...,"S Jackson and 14th Ave S, Seattle, WA ,98144.0",4,5,"(47.5537076, -122.3145837)",King County,6,Modern Streetcar,47.553708,-122.314584,"POLYGON ((-122.31097 47.55371, -122.31099 47.5..."
4,1,King County Department of Metro Transit,Full Reporter,Urban,Streetcar Rail,1853,Simple At-Grade Platform Station,FHSC: 14TH AVE S BETWEEN S WASHINGTON ST AND S...,14th Ave S and S Main St,Seattle,...,"14th Ave S and S Main St, Seattle, WA ,98144.0",4,5,"(47.6000151, -122.3141483)",King County,6,Modern Streetcar,47.600015,-122.314148,"POLYGON ((-122.31053 47.60002, -122.31055 47.5..."


In [10]:
#Identify the number of rows and columns in the new file
buffered_stations.shape

(3946, 28)